# Project - 1 : Apache Spark 

Instructor: Ramesh Yerraballi
TA: Madhumitha Sakthi
Semester: Fall 2019
Due Date: 11:59pm, Monday 9/16

This project is based on Map-Reduce Framework. In these you will get to work with Spark and will get to know how 
does spark work, what functionalities does spark provide, what does map-reduce framework do and why is it useful. 

In this project you will be implementing a basic song recommender system. You will be given a dataset where there are multiple csv files. These csv files have data corresponding to song play count and song information.

The data you would be using will be provided in a zip file along with this notebook. The __msd.zip__ archive contains '_kaggle_visible_evaluation_triplets.txt_'. We will be using the visible part of the testing data to understand the working on Apache Spark.  The user's listening history is provided as: (user, song, play count). In another file, '_kaggle_songs.txt_', each song is marked using an index for easier representation of songs. 



What to turn in:
A zip folder which will have:
- Jupyter Notebook
- A brief report on what features you used for recommendation. And a brief explanation of flow of your code. For example,  what RDD does what or, why it was created.
- datasets folder with the csv files you are using in your notebook.
- Notebook should use relative path to the csv files in datasets folder.
- Name of the zip folder - <your\_name>\_<your\_partner_name>.zip


This project consists of 4 questions. 
1. Create an RDD with _msd_evalutation_triplets.txt_ and replace the song name with the song index from _msd_songs.txt_. Identify the number of songs that do not have any rating. 
2. Generate song ratings based on the song play count as a normalized score between 0 and 1. 
3. Identify the popular song based on this rating and recommend songs to user, given user id based on the algorithm used in Movie recommender system from class. 
4. Using Cosine similarity function, identify pair-wise similarity between each pair of users and generate the top 5 most similar users without an overlap in users. 

The above list is the higer level idea about the questions. 

In [1]:
### Starter code ####

##### These lines are to tell jupyter where to find Apache Spark ####
import findspark
findspark.init('C:\\apachespark')
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Songs")
# There are two configurable parameters
# 1. A cluster URL, namely  local  in this example, which tells Spark how to connect
# to a cluster.  local  is a special value that runs Spark on one thread on the local
# machine, without connecting to a cluster.
# 2. An application name, namely  Movies  in this example. This will identify your
# application on the cluster manager’s UI if you connect to a cluster.
sc = SparkContext(conf = conf)
##### These lines are to tell jupyter where to find Apache Spark ####

In [2]:
## Read triplet file into RDD
triplet_rdd = sc.textFile(r"kaggle_visible_evaluation_triplets.txt") \
    .map(lambda line: line.split("\t")) 

## Step 1: 
Replace song name with song index and identify the number of songs without user history

In [3]:
from operator import add

In [4]:
song_rdd = sc.textFile(r"kaggle_songs.txt").map(lambda line: line.split(" "))
song_map = song_rdd.map(lambda x:(x[0],x[1]))
count = song_rdd.map(lambda x:1).reduce(add)
count
#print(song_map.take(3))

386213

In [6]:
triplet_map = triplet_rdd.map(lambda x:(x[1],(x[0],x[2])))
join0 = song_map.leftOuterJoin(triplet_map)
print(join0.take(2))
final = join0.map(lambda x:(x[1]))

print(final.take(3))
def liked(col):
    if(col[1] != None):
        return True
    return False
replace = final.filter(liked).map(lambda x:(x[1][0],x[0],x[1][1]))

#replace.collect()
print(replace.take(3))

[('SOAAALJ12AB01828B4', ('12', None)), ('SOAAAQN12AB01856D3', ('16', ('32cf63cf65787ce7e72fc7fda6ee585979af6582', '1')))]
[('12', None), ('16', ('32cf63cf65787ce7e72fc7fda6ee585979af6582', '1')), ('16', ('d1b88940eabd4fab860edf68f0f0842e4f902c78', '1'))]
[('32cf63cf65787ce7e72fc7fda6ee585979af6582', '16', '1'), ('d1b88940eabd4fab860edf68f0f0842e4f902c78', '16', '1'), ('140be58d1ea6d645ed7b317f61197b97975ac1ba', '29', '1')]


In [7]:
nore = replace.map(lambda x:(x[1],x))
noredun = nore.reduceByKey(lambda x,y:x)
count1=noredun.map(lambda x:1).reduce(add)
outcome = count-count1
print("The number of songs without user history is ", outcome )

The number of songs without user history is  223007


## Step 2:
Generate song ratings based on the play_count. For example, if (song_1, 5; song_2, 10; song_3, 5) i.e., song_1 is played 5 times, song_2 is played 10 times and song_3 is played 5 times, the normalized rating score should be 0.25, 0.5 and 0.25 respectively. 
Similarly, generate the rating for all the songs. You may notice that based on all songs, the rating is almost always very low. So, think of the best way to convert song count to ratings. (Hint: Try generating ratings based on each user's song play history)

In [8]:
#user_rdd = sc.textFile(r"kaggle_users.txt") \
 #   .map(lambda line: line.split("\n")) 
#usermap = user_rdd.map(lambda x: x[0])
# usermap.take(5)
usertimes = triplet_rdd.map(lambda x:(x[0],int(x[2]))).reduceByKey(add)
print(usertimes.take(5))
usertimes_map =usertimes.map(lambda x:(x[0],x[1]))
# times = triplet_rdd.map(lambda x:(x[1],1)).reduceByKey(add)
# print(times.take(5))
# time = triplet_rdd.map(lambda x:(x[1],int(x[2]))).reduceByKey(add)
# print(time.take(5))

[('d7083f5e1d50c264277d624340edaaf3dc16095b', 17), ('d68dc6fc25248234590d7668a11e3335534ae4b4', 6), ('fdf6afb5daefb42774617cf223475c6013969724', 10), ('10cbcd627472477dfbec90fb75017f8df6ce84ec', 9), ('30e4a688e6fc9c8bfe55998af3996a909ae34449', 8)]


In [10]:
replace_map = replace.map(lambda x:(x[0],(x[1],x[2])))
join1 = usertimes_map.leftOuterJoin(replace_map)
# print(join1.take(3))
ratings = join1.map(lambda x:(x[0],x[1][1][0],int(x[1][1][1])/x[1][0]))
print(ratings.take(5))
# join1 = times.leftOuterJoin(time)
# # print(join1.take(3))
# ratings = join1.map(lambda x:(x[0],x[1][1]/x[1][0]))
# # print(ratings.take(3))
# ratings.collect()
#ratings.collect()

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', '36681', 0.11363636363636363), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '75497', 0.18181818181818182), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '245936', 0.06818181818181818), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '383719', 0.022727272727272728), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '129546', 0.1590909090909091)]


In [11]:
import math
final_rating = ratings.map(lambda x:(x[0],x[1],10+math.log(x[2])))
#final_rating.collect()
print(final_rating.take(5))

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', '36681', 7.825248278515839), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '75497', 8.295251907761575), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '245936', 7.3144226547498485), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '383719', 6.2158103660817385), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', '129546', 8.161720515137052)]


## Step 3: 
For a given user_id, rating, recommend 5 other songs from the list. One way to do this is based on another user who liked the same song liked by this user with rating more than the given rating and recommend the 5 songs based on the matched user's rating. 

In [ ]:
# triplet_map1 = triplet_rdd.map(lambda x:(x[1],(x[0],x[2])))
# ratings_map = ratings.map(lambda x:(x[0],x[1]))
# join2 = triplet_map1.leftOuterJoin(ratings_map)
# join2.take(5)
# def liked(col):
#     if(col[1] != None and col[0] != None):
#         return True
#     return False
# whole = join2.filter(liked).map(lambda x:(x[1][0][0],x[0],x[1][1],x[1][0][1]))

# whole.take(5)

# #hi :)

In [12]:
## Take inputs from final ratings from step 2
givenSongId = '36681'; # which song
givenUserId = '6530c4fc41b9110de5d39fe0355fa103c66385f0'; # which User
givenRating = 7.825248278515839; # what is the threshold for "liking"

In [14]:
##    userId, songId and rating
final_collect = final_rating.map(lambda line: (line[0], [line[1], line[2]]))
####################################################
#final_collect.collect()
print(final_collect.take(5))

[('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['36681', 7.825248278515839]), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['75497', 8.295251907761575]), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['245936', 7.3144226547498485]), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['383719', 6.2158103660817385]), ('6530c4fc41b9110de5d39fe0355fa103c66385f0', ['129546', 8.161720515137052])]


In [21]:
## Function to filter users who also liked the given song
def liked(col):
    if((col[0] != '') and (col[1][0] == givenSongId) and (float(col[1][1]) > givenRating)):
        return True
    return False
####################################################
## Get all users who liked the given song and what rating they gave it
## above the like threshold
users = final_collect.filter(liked) \
                    .map(lambda line: (line[0], line[1][1])) 
users.collect()

[('6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945),
 ('af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659),
 ('80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 8.208240530771945),
 ('b43f79fc00f1ff69eb272aa40721c3a1ec7ba911', 7.985096979457735),
 ('70349986733e95f7ca4ce65daa575d673fd26109', 8.054089850944687),
 ('e7a3c5907d77e3f03c2adc9ea168404d8a891a04', 8.341771923396468),
 ('eacd33bf44d39f537591fb06aad0b28dd48f980c', 8.32602356642833),
 ('7c948aa17a6528c319eed401f3559d862032910c', 7.871768294150732),
 ('fbcc48dfceb317106323ad9b959b0e9d98be52ff', 9.106182123977904),
 ('194396fa0779016224bca8503b6ac5eb4617cc2c', 8.495922603223725),
 ('901e60fdedebb03b0b7714719d48f560eb2367e6', 7.920558458320164),
 ('7c71b8809696fc199305bc71b49b0d2ff0b56d54', 8.3905620875659),
 ('42f4f6585483ef3a7fab1314a8b1808236033615', 8.208240530771945),
 ('e512f0042cc25d560e23d3760f024271515a89d4', 8.424463639241582),
 ('183e5fa03bf810c12e9968be67c7657440ab4235', 8.083077387817939),
 ('b52ffe126320

In [22]:
## Get songs that users who liked the given song, listened to
## (songId, [userId, rating])
usersSongs = final_collect.join(users) \
                .map(lambda line: (line[1][0][0], [line[0], line[1][0][1]]))
####################################################
usersSongs.collect()

[('36681', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('288097', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('385781', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('187518', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('337171', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.90138771133189]),
 ('36681', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('340724', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('172659', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('281084', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('35213', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('36681', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 8.208240530771945]),
 ('208185', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 7.109628242103835]),
 ('208210', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 7.109628242103835]),
 

In [23]:
## Function to filter not to keep the given song
def notTheGivenSong(row):
    if(row[0] != givenSongId):
        return True
    return False
####################################################
## Next step: Get only those songs that are liked by the users who liked given song
## and their aggregate rating
usersSongsWithoutGiven = usersSongs.filter(notTheGivenSong)
####################################################
usersSongsWithoutGiven.collect()

[('288097', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('385781', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('187518', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.208240530771945]),
 ('337171', ['6385fa76b90e95f49909701b5effcdc17ea3bc0a', 8.90138771133189]),
 ('340724', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('172659', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('281084', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('35213', ['af71c172969f8918b58b7d160177e5e332332bc5', 8.3905620875659]),
 ('208185', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 7.109628242103835]),
 ('208210', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 7.109628242103835]),
 ('245050', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 8.495922603223725]),
 ('287839', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 7.109628242103835]),
 ('303151', ['80b0aeeb14f49454ad61f5d3b54460aacb0e6cbe', 7.109628242103835

In [25]:
#Drop the users from RDD
noSongsInUsers = usersSongsWithoutGiven.map (lambda x: (x[0], float (x[1][1])))
noSongsInUsers.collect()

[('288097', 8.208240530771945),
 ('385781', 8.208240530771945),
 ('187518', 8.208240530771945),
 ('337171', 8.90138771133189),
 ('340724', 8.3905620875659),
 ('172659', 8.3905620875659),
 ('281084', 8.3905620875659),
 ('35213', 8.3905620875659),
 ('208185', 7.109628242103835),
 ('208210', 7.109628242103835),
 ('245050', 8.495922603223725),
 ('287839', 7.109628242103835),
 ('303151', 7.109628242103835),
 ('352646', 7.109628242103835),
 ('377773', 7.109628242103835),
 ('144962', 7.80277542266378),
 ('199054', 7.109628242103835),
 ('206245', 7.109628242103835),
 ('302128', 7.109628242103835),
 ('272257', 6.955477562276577),
 ('373359', 6.955477562276577),
 ('265906', 8.564915474710677),
 ('366243', 6.955477562276577),
 ('50702', 6.955477562276577),
 ('71156', 6.955477562276577),
 ('153916', 6.955477562276577),
 ('39077', 6.955477562276577),
 ('48941', 8.054089850944687),
 ('168176', 7.648624742836523),
 ('60860', 7.920558458320164),
 ('64432', 7.227411277760218),
 ('15718', 7.227411277760

In [26]:
#Getting the song indexes with top 5 aggregated ratings
aggSongs = noSongsInUsers.reduceByKey(add)
aggSongs.takeOrdered(5, key= lambda x: -x[1])

[('328524', 23.75002475774052),
 ('215861', 15.030186700423998),
 ('286747', 14.96304739758637),
 ('36622', 14.36521039683075),
 ('242113', 13.750990474860476)]

In [28]:
##Reordering the songs by switching index and song name
song_names_reorderedd = song_rdd.map(lambda x: (x[1], x[0]))
song_names_reorderedd.take(4)

[('1', 'SOAAADD12AB018A9DD'),
 ('2', 'SOAAADE12A6D4F80CC'),
 ('3', 'SOAAADF12A8C13DF62'),
 ('4', 'SOAAADZ12A8C1334FB')]

In [32]:
#Do a join of the aggregated songs so now we have song index, aggregated ratings, and song name
song_names = aggSongs.join(song_names_reorderedd)
song_names.take(5)

[('281084', (8.3905620875659, 'SOSJRXV12A8C136E1B')),
 ('123485', (7.227411277760218, 'SOHTEDD12A6D4F8215')),
 ('242113', (13.750990474860476, 'SOPQJAZ12A6310F168')),
 ('225526', (8.054089850944687, 'SOOMNQN12AB0185E62')),
 ('71156', (6.955477562276577, 'SOELASR12A6D4FAB10'))]

In [33]:
#Reordering song names so now we have song name, aggregated ratings
song_names_reformatted = song_names.map(lambda x: (x[1][1],x[1][0]))
song_names_reformatted.take(5)

[('SOSJRXV12A8C136E1B', 8.3905620875659),
 ('SOHTEDD12A6D4F8215', 7.227411277760218),
 ('SOPQJAZ12A6310F168', 13.750990474860476),
 ('SOOMNQN12AB0185E62', 8.054089850944687),
 ('SOELASR12A6D4FAB10', 6.955477562276577)]

In [34]:
#Take top 5 from song names list
song_names_reformatted.takeOrdered(5, key= lambda x: -x[1])

[('SOVTLQW12AB0186641', 23.75002475774052),
 ('SONVPTP12A6D4F7A34', 15.030186700423998),
 ('SOSUADS12A58A80A47', 14.96304739758637),
 ('SOCGXXL12B0B808865', 14.36521039683075),
 ('SOPQJAZ12A6310F168', 13.750990474860476)]

## Step 4: 
1. Compute cosine similarity between all pairs of users. 
2. Sort the similarity score and print the top-5 similar users. 
3. If the top-5 user set has an user appearing more than once, ignore that pair and take the next best pair from the sorted list. 
4. For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list. 

In [ ]:
#1.Compute cosine similarity between all pairs of users.
